# Anatomical Preprocessing

This notebooks preprocesses anatomical MRI images by executing the following processing steps:

1. Reorient images to RAS
1. Crop FOV with FSL
1. N4-inhomogenity correction with ANTS
1. GM, WM and CSF segmentation with SPM
1. Brainmask creation and brain extraction with Nilearn
1. Normalization to ICBM template with ANTS

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format:

    dataset
    ├── analysis-anat_specs.json
    └── sub-{sub_id}
        └── anat
            └── sub-{sub_id}*{T1_id}*.nii.gz
            
**Note:** Subfolders for individual scan sessions are optional. `fmriflows` will run the preprocessing on all files of a subject.

## Execution Specifications

This notebook will extract the relevant processing specifications from the `analysis-anat_specs.json` file in the dataset folder. In the current setup, they are as follows:

In [ ]:
import json
from os.path import join as opj

spec_file = opj('/data', 'analysis-anat_specs.json')

with open(spec_file) as f:
    specs = json.load(f)

specs

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject names
subject_list = specs['subject_list']

# Anatomical image identifier
T1_id = specs['T1_id']

# Resolution of normalized images
norm_res = specs['vox_res']

# Number of parallel jobs to run
n_proc = specs['n_parallel_jobs']

# Create the Workflow

## Import Modules

In [ ]:
from os.path import join as opj
from nipype import Node, Workflow, Function, IdentityInterface
from nipype.interfaces.image import Reorient
from nipype.interfaces.fsl import RobustFOV
from nipype.interfaces.ants import N4BiasFieldCorrection, Registration
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.spm import NewSegment
from nipype.interfaces.io import SelectFiles, DataSink

In [ ]:
# Specify SPM location
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-dev/spm12_mcr/spm/spm12')

## Relevant Execution Variables

In [ ]:
# Folder paths and names
exp_dir = '/data/derivatives'
out_dir = 'fmriflows'
work_dir = '/workingdir'

In [ ]:
# Create fmriflows output folder if missing
import pathlib
pathlib.Path(opj(exp_dir, out_dir)).mkdir(parents=True, exist_ok=True) 

In [ ]:
# Creation of template brain with desired voxel resolution
template_dir = '/templates/mni_icbm152_nlin_asym_09c/'
brain_template = opj(template_dir, '1.0mm_brain.nii.gz')

In [ ]:
# Resample template brain to desired resolution
from nibabel import load, Nifti1Image
from nilearn.image import resample_img
from nibabel.spaces import vox2out_vox

img = load(brain_template)
target_shape, target_affine = vox2out_vox(img, voxel_sizes=norm_res)
img_resample = resample_img(img, target_affine, target_shape, clip=True)
norm_template = opj(template_dir, 'template_brain.nii.gz')
img_resample.to_filename(norm_template)

## Implement Nodes

In [ ]:
# Reorient anatomical images to RAS
reorient = Node(Reorient(orientation='RAS'), name='reorient')

In [ ]:
# Reduces FOV of images to remove lower head and neck
cropFOV = Node(RobustFOV(output_type='NIFTI_GZ'), name='cropFOV')

In [ ]:
# Corrects bias field
n4 = Node(N4BiasFieldCorrection(dimension=3), name='n4')

In [ ]:
# Gunzips images
gunzip = Node(Gunzip(), name='gunzip')

In [ ]:
# Segments brain into 5 classes (GM, WM, CSF, Skull & Head)
segment = Node(NewSegment(), name='segment')

In [ ]:
# Compute Brain Mask and Extract Brain
def get_brain_and_mask(in_file, segments):
    
    import nibabel as nb
    from nilearn.image import clean_img, mean_img, math_img
    from scipy.ndimage.morphology import (
        binary_fill_holes, binary_dilation, binary_erosion)

    # Load T1w corrected image
    img = nb.load(in_file)

    # Brainmask is created from the probability tissue maps
    gm, wm, csf, skull, head = [s[0] for s in segments]
    img_gmwm = math_img("(img1 + img2) >= 0.25", img1=gm, img2=wm)
    img_csf = math_img("img1 >= 1.0", img1=csf)
    img_not_rest = math_img("(img1 + img2) >= 0.25", img1=head, img2=skull)
    img_mask = math_img("(img1 + img2 - img3) >= 1.0", img1=img_gmwm, img2=img_csf, img3=img_not_rest)

    # Improves brainmask by 1 x erosion, 2 x dilation & filling of wholes
    data_mask = binary_erosion(
                binary_fill_holes(
                binary_dilation(
                img_mask.get_data(),
                    iterations = 2)),
                    iterations = 1).astype('int8')
    img_mask = nb.Nifti1Image(data_mask, img.affine, img.header)

    # Extract Brain with Mask
    img_brain = math_img("img1 * img2", img1=img, img2=img_mask)

    # Store output in files
    out_file = in_file.replace('.nii', '_brain.nii')
    mask = in_file.replace('.nii', '_brainmask.nii')
    img_brain.to_filename(out_file)
    img_mask.to_filename(mask)

    return out_file, mask

extract_brain = Node(Function(input_names=['in_file', 'segments'],
                              output_names=['out_file', 'mask'],
                              function=get_brain_and_mask),
                     name='extract_brain')

In [ ]:
# Normalize anatomy to ICBM template
antsreg = Node(Registration(fixed_image=norm_template,
                            num_threads=n_proc,
                            output_inverse_warped_image=True,
                            output_warped_image=True,

                            collapse_output_transforms=True,
                            dimension=3,
                            float=True,
                            initial_moving_transform_com=True,
                            initialize_transforms_per_stage=False,
                            interpolation='LanczosWindowedSinc',
                            transforms=['Rigid', 'Affine', 'SyN'],
                            transform_parameters=[(0.05,), (0.08,),
                                                  (0.1, 3.0, 0.0)],

                            metric=['Mattes', 'Mattes', 'CC'],
                            metric_weight=[1.0] * 3,
                            radius_or_number_of_bins=[56, 56, 4],
                            sampling_strategy=['Regular', 'Regular', 'None'],
                            sampling_percentage=[0.25, 0.25, 1],
                            number_of_iterations=[[100, 100],
                                                  [100, 100],
                                                  [100, 50, 20]],
                            convergence_threshold=[1e-06] * 3,
                            convergence_window_size=[20, 20, 10],
                            smoothing_sigmas=[[2, 1], [1, 0], [3, 2, 1]],
                            sigma_units=['vox'] * 3,
                            shrink_factors=[[2, 1], [2, 1], [8, 4, 2]],
                            use_estimate_learning_rate_once = [True ,True, True],
                            use_histogram_matching=True,

                            winsorize_lower_quantile=0.005,
                            winsorize_upper_quantile=0.995,
                            write_composite_transform=True,
                            terminal_output='file'),
               name='antsreg')

In [ ]:
# Create Visual Report for Anatomical Preprocessing
def write_report(sub, sess, n4, segments, brain, T1_template, warped_file):
    
    import os
    import nibabel as nb
    from nilearn.plotting import plot_stat_map, plot_roi
    from matplotlib.pyplot import figure

    import numpy as np
    from nilearn.image import coord_transform
    
    # Support Function to get optimal cut for visualization
    def get_cut_ids(img, axis=0):

        # Compute voxel id to cut
        idx = np.sort(img.get_data().nonzero()[axis])
        vox_id = np.linspace(idx.min(), idx.max(), num=12, endpoint=True).astype('int')
        vox_id = vox_id[2:-2]

        # Translate voxel id to image space
        if axis == 0:
            cut_ids = [int(coord_transform(r, 0, 0, img.affine)[0]) for r in vox_id]
        elif axis == 1:
            cut_ids = [int(coord_transform(0, r, 0, img.affine)[1]) for r in vox_id]
        elif axis == 2:
            cut_ids = [int(coord_transform(0, 0, r, img.affine)[2]) for r in vox_id]
        return cut_ids
    
    title_txt = 'sub: %s' % sub
    
    # Add session suffix if present
    if sess:
        title_txt += ' - sess: %s' % sess
    
    # Visualize Tissue Segmentation of T1w
    img = nb.load(brain)
    data = np.stack((np.zeros(img.get_data().shape),
                     nb.load(segments[0][0]).get_data(),
                     nb.load(segments[1][0]).get_data(),
                     nb.load(segments[2][0]).get_data(),
                     nb.load(segments[3][0]).get_data(),
                     nb.load(segments[4][0]).get_data()), axis= -1)
    label_id = np.argmax(data, axis=-1)
    segmentation = nb.Nifti1Image(label_id, img.affine, img.header)

    fig = figure(figsize=(16, 8))
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)

        plot_roi(segmentation, cmap='Accent', dim=1, annotate=False, bg_img=n4,
                 display_mode=e, title=title_txt + ' - %s-axis' % e,
                 resampling_interpolation='nearest',
                 cut_coords=get_cut_ids(img, i), axes=ax)
    
    out_segmentation = brain.replace('brain.nii.gz', 'segmentation.svg')
    fig.savefig(out_segmentation, bbox_inches='tight', facecolor='black',
                frameon=True, dpi=300, transparent=True)

    # Visualize Brain Extraction of T1w
    fig = figure(figsize=(16, 8))
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)
        plot_stat_map(brain, title=title_txt + ' - %s-axis' % e, colorbar=False,
                      threshold='auto', bg_img=n4, cmap='magma', display_mode=e,
                      resampling_interpolation='nearest', dim=-1,
                      cut_coords=get_cut_ids(nb.load(brain), i), annotate=False, axes=ax)

    out_brain = brain.replace('.nii.gz', '.svg')
    fig.savefig(out_brain, bbox_inches='tight', facecolor='black', frameon=True,
                dpi=300, transparent=True)
    
    # Visualize T1w to MNI registration
    fig = figure(figsize=(16, 8))
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)
        plot_stat_map(warped_file, title=title_txt + ' - %s-axis' % e, colorbar=False,
                      threshold='auto', bg_img=T1_template, display_mode=e,
                      resampling_interpolation='nearest',
                      cut_coords=get_cut_ids(nb.load(warped_file), i),
                      cmap='magma', annotate=False, axes=ax)
    
    out_warp = warped_file.replace('.nii.gz', '.svg')
    fig.savefig(out_warp, bbox_inches='tight', facecolor='black', frameon=True,
                dpi=300, transparent=True)
    
    # Write the HTML report
    with open('/templates/report_template.html', 'r') as report:
        txt = report.read()
        txt = txt.replace('sub-placeholder', 'sub-%s' % sub)
        
        # Add session suffix if present
        if sess:
            txt = txt.replace('ses-placeholder', 'ses-%s' % sess)
            filename = 'sub-%s_ses-%s.html' % (sub, sess)
        else:
            txt = txt.replace('ses-placeholder', '')
            txt = txt.replace('__', '_')
            filename = 'sub-%s.html' % sub

    report_file = os.path.join('/data', 'derivatives', 'fmriflows', filename)
    
    with open(report_file, 'w') as report:
        report.writelines(txt)

    return out_segmentation, out_brain, out_warp
    
# Create Report Node
create_report = Node(Function(input_names=['sub', 'sess', 'n4', 'segments', 'brain',
                                           'T1_template', 'warped_file'],
                              output_names=['out_segmentation', 'out_brain', 'out_warp'],
                              function=write_report),
                name='create_report')
create_report.inputs.T1_template = brain_template.replace('brain', 'T1')

## Specify Input & Output Stream

In [ ]:
# Get all anatomical files
from bids.grabbids import BIDSLayout
layout = BIDSLayout('/data/')

In [ ]:
# Get session name if it exists
session_list = layout.get_sessions()
session_list = session_list if session_list else ['']

In [ ]:
# Iterate over subject and session id
infosource = Node(IdentityInterface(fields=['subject_id', 'session_id']),
                  name='infosource')
infosource.iterables = [('subject_id', subject_list),
                        ('session_id', session_list)]

In [ ]:
# Compute Brain Mask and Extract Brain
def create_file_path(subject_id, session_id, layout, T1_id):

    from os.path import join
    
    entities = {'subject_id': subject_id,
                'T1_id': T1_id}
    
    # Add session id if present in dataset
    if session_id != '':
        entities['session_id'] = session_id
    
    pattern = 'sub-{subject_id}[/ses-{session_id}]/anat/'
    pattern += 'sub-{subject_id}[_ses-{session_id}]_{T1_id}.nii.gz'

    fpath = layout.build_path(entities, path_patterns=[pattern])

    return join('/data', fpath)

selectfiles = Node(Function(input_names=['subject_id', 'session_id',
                                         'layout', 'T1_id'],
                            output_names=['anat'],
                            function=create_file_path),
                   name='selectfiles')
selectfiles.inputs.layout = layout
selectfiles.inputs.T1_id = T1_id

In [ ]:
# Save relevant outputs in a datasink
datasink = Node(DataSink(base_directory=exp_dir,
                         container=out_dir),
                name='datasink')

In [ ]:
# Apply the following naming substitutions for the datasink
substitutions = [('_session_id_%s_subject_id_%s/' % (sess, sub),
                  'sub-%s/sub-%s_ses-%s_' % (sub, sub, sess))
                 for sess in session_list
                 for sub in subject_list]
substitutions += [('_ras', ''),
                  ('_ROI', ''),
                  ('_%s_corrected' % T1_id, ''),
                  ('c1', 'seg_gm_'),
                  ('c2', 'seg_wm_'),
                  ('c3', 'seg_csf_'),
                  ('c4', 'seg_skull_'),
                  ('c5', 'seg_head_'),
                  ('ses-_', ''),
                 ]
substitutions += [('sub-%s_sub-%s' % (sub, sub), 'sub-%s' % sub)
                  for sub in subject_list]
substitutions += [('_sub-%s_ses-%s' % (sub, sess), '')
                  for sess in session_list
                  for sub in subject_list]
substitutions += [('_sub-%s.nii' % sub, '.nii')
                  for sub in subject_list]
substitutions += [('/sub-%s_ses-%s.nii' % (sub, sess),
                   '/sub-%s_ses-%s_T1w_corrected.nii' % (sub, sess))
                  for sess in session_list
                  for sub in subject_list]
substitutions += [('/sub-%s.nii' % sub,
                   '/sub-%s_T1w_corrected.nii' % sub)
                  for sess in session_list
                  for sub in subject_list]
datasink.inputs.substitutions = substitutions

## Create Preprocessing Workflow

In [ ]:
# Create anatomical preprocessing workflow
preproc_anat = Workflow(name='preproc_anat')
preproc_anat.base_dir = work_dir

In [ ]:
# Add nodes to workflow and connect them
preproc_anat.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                                 ('session_id', 'session_id')]),

                      # Main part of workflow
                      (selectfiles, reorient, [('anat', 'in_file')]),
                      (reorient, cropFOV, [('out_file', 'in_file')]),
                      (cropFOV, n4, [('out_roi', 'input_image')]),
                      (n4, gunzip, [('output_image', 'in_file')]),
                      (gunzip, segment, [('out_file', 'channel_files')]),
                      (segment, extract_brain, [('native_class_images', 'segments')]),
                      (n4, extract_brain, [('output_image', 'in_file')]),
                      (extract_brain, antsreg, [('out_file', 'moving_image')]),

                      # Store main results in datasink
                      (n4, datasink, [('output_image', 'preproc_anat.@n4')]),
                      (segment, datasink, [
                          ('native_class_images', 'preproc_anat.@segment')]),
                      (extract_brain, datasink, [('out_file', 'preproc_anat.@brain'),
                                                 ('mask', 'preproc_anat.@mask')]),
                      (antsreg, datasink, [
                          ('warped_image', 'preproc_anat.@warped_image'),
                          ('inverse_warped_image', 'preproc_anat.@inverse_warped_image'),
                          ('composite_transform', 'preproc_anat.@transform'),
                          ('inverse_composite_transform', 'preproc_anat.@inverse_transform')]),

                      # Create visual report
                      (infosource, create_report, [('subject_id', 'sub'),
                                                   ('session_id', 'sess')]),
                      (n4, create_report, [('output_image', 'n4')]),
                      (segment, create_report, [('native_class_images', 'segments')]),
                      (extract_brain, create_report, [('out_file', 'brain')]),
                      (antsreg, create_report, [('warped_image', 'warped_file')]),
                      (create_report, datasink, [('out_segmentation', 'preproc_anat.@vis_segmentation'),
                                                 ('out_brain', 'preproc_anat.@vis_brain'),
                                                 ('out_warp', 'preproc_anat.@vis_warp')]),
                      ])

## Visualize Workflow

In [ ]:
# Create preproc_anat output graph
preproc_anat.write_graph(graph2use='colored', format='svg', simple_form=True)

# Visualize the graph in the notebook
from IPython.display import SVG
SVG(filename=opj(preproc_anat.base_dir, 'preproc_anat', 'graph.svg'))

# Run Workflow

In [ ]:
# Run the workflow in sequential mode
preproc_anat.run('Linear')

In [ ]:
# Save workflow graph visualizations in datasink
preproc_anat.write_graph(graph2use='flat', format='svg', simple_form=True)
preproc_anat.write_graph(graph2use='colored', format='svg', simple_form=True)

from shutil import copyfile
copyfile(opj(preproc_anat.base_dir, 'preproc_anat', 'graph.svg'),
         opj(exp_dir, out_dir, 'preproc_anat', 'graph.svg'))
copyfile(opj(preproc_anat.base_dir, 'preproc_anat', 'graph_detailed.svg'),
         opj(exp_dir, out_dir, 'preproc_anat', 'graph_detailed.svg'));